In [1]:
import os
import urllib.request
import tarfile
import scipy.io
import shutil

# URLs
IMG_URL   = 'http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz'
LBL_URL   = 'http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat'

# Paths
RAW_DIR   = './data/flowers_raw'
TGZ_PATH  = os.path.join(RAW_DIR, '102flowers.tgz')
MAT_PATH  = os.path.join(RAW_DIR, 'imagelabels.mat')
EXTRACTED = os.path.join(RAW_DIR, 'jpg')
DST_DIR   = './data/flowers'

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(DST_DIR, exist_ok=True)

# Download
urllib.request.urlretrieve(IMG_URL, TGZ_PATH)
urllib.request.urlretrieve(LBL_URL, MAT_PATH)

# Extract images
with tarfile.open(TGZ_PATH, 'r:gz') as tar:
    tar.extractall(path=RAW_DIR)

# Load labels
mat    = scipy.io.loadmat(MAT_PATH)
labels = mat['labels'][0]   # array of length 8189, values 1–102

# Organize into class subfolders
for idx, cls in enumerate(labels, start=1):
    cls_folder = os.path.join(DST_DIR, f"{cls:03d}")
    os.makedirs(cls_folder, exist_ok=True)
    src_img = os.path.join(EXTRACTED, f"image_{idx:05d}.jpg")
    dst_img = os.path.join(cls_folder, f"image_{idx:05d}.jpg")
    shutil.copy(src_img, dst_img)

print("Data prep complete. Structured data in:", DST_DIR)

Data prep complete. Structured data in: ./data/flowers


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, utils
from torch.utils.data import DataLoader

# ——————————————— Config ———————————————
data_dir    = './data/flowers'    # output of step 2
out_dir     = './outputs'
image_size  = 64
batch_size  = 64
z_dim       = 100
ngf, ndf    = 64, 64
num_epochs  = 50
lr          = 2e-4
beta1       = 0.5
device      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

os.makedirs(out_dir, exist_ok=True)

# ——————————————— DataLoader ———————————————
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])
dataset = datasets.ImageFolder(data_dir, transform=transform)
loader  = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [3]:
# —————————————— Weight init ——————————————
def weights_init(m):
    classname = m.__class__.__name__
    if 'Conv' in classname:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif 'BatchNorm' in classname:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [4]:
# ————————————— Generator —————————————
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(z_dim, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8), nn.ReLU(True),
            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4), nn.ReLU(True),
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2), nn.ReLU(True),
            nn.ConvTranspose2d(ngf*2, ngf,   4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),   nn.ReLU(True),
            nn.ConvTranspose2d(ngf, 3,       4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, x):
        return self.main(x)

In [5]:
# ————————————— Discriminator —————————————
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*8), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.main(x).view(-1)

In [6]:
# ————————— Instantiate —————————
netG = Generator().to(device); netG.apply(weights_init)
netD = Discriminator().to(device); netD.apply(weights_init)
criterion   = nn.BCELoss()
optD        = optim.Adam(netD.parameters(), lr=lr, betas=(beta1,0.999))
optG        = optim.Adam(netG.parameters(), lr=lr, betas=(beta1,0.999))
fixed_noise = torch.randn(64, z_dim, 1, 1, device=device)

In [7]:
# ——————————— Training Loop ———————————
for epoch in range(1, num_epochs+1):
    for i, (real, _) in enumerate(loader, 1):
        b_size = real.size(0)
        real, labels_real = real.to(device), torch.ones(b_size, device=device)
        labels_fake       = torch.zeros(b_size, device=device)

        # Train Discriminator
        netD.zero_grad()
        out_real = netD(real)
        lossD_real = criterion(out_real, labels_real)
        noise     = torch.randn(b_size, z_dim, 1, 1, device=device)
        fake_imgs = netG(noise)
        out_fake  = netD(fake_imgs.detach())
        lossD_fake= criterion(out_fake, labels_fake)
        (lossD_real + lossD_fake).backward()
        optD.step()

        # Train Generator
        netG.zero_grad()
        out2 = netD(fake_imgs)
        lossG= criterion(out2, labels_real)
        lossG.backward()
        optG.step()

        if i % 100 == 0:
            print(f"[{epoch}/{num_epochs}][{i}/{len(loader)}] "
                  f"Loss_D: {(lossD_real+lossD_fake).item():.4f}  Loss_G: {lossG.item():.4f}")

    # save sample grid
    with torch.no_grad():
        grid = utils.make_grid(netG(fixed_noise).cpu(), padding=2, normalize=True)
        utils.save_image(grid, f"{out_dir}/epoch_{epoch:03d}.png")

[1/50][100/128] Loss_D: 0.3584  Loss_G: 6.9421
[2/50][100/128] Loss_D: 0.2060  Loss_G: 4.0963
[3/50][100/128] Loss_D: 0.3166  Loss_G: 6.1467
[4/50][100/128] Loss_D: 0.3276  Loss_G: 5.2761
[5/50][100/128] Loss_D: 0.8813  Loss_G: 5.8841
[6/50][100/128] Loss_D: 0.4318  Loss_G: 5.6299
[7/50][100/128] Loss_D: 1.0002  Loss_G: 1.8013
[8/50][100/128] Loss_D: 0.9953  Loss_G: 4.7000
[9/50][100/128] Loss_D: 0.6727  Loss_G: 3.5765
[10/50][100/128] Loss_D: 0.7395  Loss_G: 2.2092
[11/50][100/128] Loss_D: 0.5242  Loss_G: 3.6913
[12/50][100/128] Loss_D: 0.4808  Loss_G: 3.2172
[13/50][100/128] Loss_D: 1.8008  Loss_G: 1.4882
[14/50][100/128] Loss_D: 0.7996  Loss_G: 1.6790
[15/50][100/128] Loss_D: 0.8850  Loss_G: 1.5200
[16/50][100/128] Loss_D: 1.5278  Loss_G: 4.0439
[17/50][100/128] Loss_D: 0.8775  Loss_G: 2.5257
[18/50][100/128] Loss_D: 0.9896  Loss_G: 4.8267
[19/50][100/128] Loss_D: 1.0011  Loss_G: 1.4251
[20/50][100/128] Loss_D: 2.1695  Loss_G: 1.6660
[21/50][100/128] Loss_D: 0.6394  Loss_G: 3.7231
[

In [8]:
# ————— Generate new images for augmentation —————
netG.eval()
n_new = 500
z     = torch.randn(n_new, z_dim, 1, 1, device=device)
with torch.no_grad():
    gen = netG(z).cpu()

aug_dir = os.path.join(out_dir, 'augmented')
# Create a subdirectory for the augmented images (a single class folder)
aug_class_dir = os.path.join(aug_dir, '000') # Using '000' as a placeholder class
os.makedirs(aug_class_dir, exist_ok=True)

for idx, img in enumerate(gen, 1):
    # Save the image inside the class subdirectory
    utils.save_image(img, f"{aug_class_dir}/aug_{idx:04d}.png", normalize=True)

print("Done! Augmented images in:", aug_class_dir) # Updated print statement

Done! Augmented images in: ./outputs/augmented/000


In [9]:
import os
import shutil

# Path to the folder you just filled with images
aug_dir = os.path.join(out_dir, 'augmented')

# Path (without extension) for the zip archive
zip_base = os.path.join(out_dir, 'augmented_images')

# This will create 'augmented_images.zip' in your out_dir
shutil.make_archive(zip_base, 'zip', root_dir=aug_dir)

print(f"Augmented images zipped at {zip_base}.zip")

Augmented images zipped at ./outputs/augmented_images.zip
